In [1]:
def split_revealed_targets(revealed_targets_df):
    # Split DataFrame based on the 'is_consumption' column
    revealed_targets_production_df = revealed_targets_df[revealed_targets_df['is_consumption'] == False].copy()
    revealed_targets_consumption_df = revealed_targets_df[revealed_targets_df['is_consumption'] == True].copy()

    # Change column names for production DataFrame
    revealed_targets_production_df.rename(columns={'target': 'target_production'}, inplace=True)

    # Change column names for consumption DataFrame
    revealed_targets_consumption_df.rename(columns={'target': 'target_consumption'}, inplace=True)

    return revealed_targets_production_df, revealed_targets_consumption_df


In [2]:
class ForecastWeatherProcessor:
    def __init__(self, forecast_weather_df, points_data_file='/kaggle/input/mapping/points_data.parquet'):
        self.forecast_weather_df = forecast_weather_df
        self.points_data_file = points_data_file
        self.county_mapping = {
            "Harjumaa": 0,
            "Hiiumaa": 1,
            "Ida-Virumaa": 2,
            "Järvamaa": 3,
            "Jõgevamaa": 4,
            "Lääne-Virumaa": 5,
            "Läänemaa": 6,
            "Pärnumaa": 7,
            "Põlvamaa": 8,
            "Raplamaa": 9,
            "Saaremaa": 10,
            "Tartumaa": 11,
            None: 12,
            "Valgamaa": 13,
            "Viljandimaa": 14,
            "Võrumaa": 15,
        }
        self.read_points_gdf = pd.read_parquet(self.points_data_file)
        self.filtered_forecast_df = None
        self.non_filtered_forecast_df = None

    def map_county_id(self):
        # Convert 'latitude' and 'longitude' to float32
        self.read_points_gdf[['latitude', 'longitude']] = self.read_points_gdf[['latitude', 'longitude']].astype('float32')

        # Map county IDs
        self.read_points_gdf['county'] = self.read_points_gdf['county'].map(self.county_mapping)

        # Merge on 'latitude', 'longitude', and convert them to float32
        self.forecast_weather_df = pd.merge(
            self.forecast_weather_df,
            self.read_points_gdf[['latitude', 'longitude', 'county']].astype('float32'),
            on=['latitude', 'longitude'],
            how='left'
        )
    def filter_forecast_data(self):
        self.forecast_weather_df.sort_values(by='origin_datetime', inplace=True)
        unique_combinations = self.forecast_weather_df.groupby(['latitude', 'longitude', 'forecast_datetime']).cumcount() == 0
        self.filtered_forecast_df = self.forecast_weather_df[unique_combinations]

    def generate_non_filtered_forecast(self):
        original_df = self.forecast_weather_df
        non_filtered_df = original_df.merge(self.filtered_forecast_df,
                                            on=['latitude', 'longitude', 'forecast_datetime', 'origin_datetime'],
                                            how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
        self.non_filtered_forecast_df = non_filtered_df

    def calculate_mean_for_filtered(self):
        grouped_df = self.filtered_forecast_df.groupby(['county', 'origin_datetime', 'forecast_datetime']).mean().reset_index()
        self.filtered_forecast_df = grouped_df

    def modify_column_names_for_filtered(self):
        self.filtered_forecast_df.columns = [col + "_f1" if col != 'county' else col for col in self.filtered_forecast_df.columns]

    def change_column_names_for_filtered(self):
        self.filtered_forecast_df.rename(columns={'county_f1': 'county', 'forecast_datetime_f1': 'datetime'}, inplace=True)
   
    def modify_column_names_for_non_filtered(self): 
        columns_to_drop = [col for col in self.non_filtered_forecast_df.columns if col.endswith('_y')]
        self.non_filtered_forecast_df.drop(columns=columns_to_drop, inplace=True)
        self.non_filtered_forecast_df.columns = self.non_filtered_forecast_df.columns.str.rstrip('_x')
   
    def change_column_names_for_non_filtered(self):
        self.non_filtered_forecast_df.rename(columns={'forecast_datetime_f2': 'datetime'}, inplace=True)
   
    
    def add_f1_and_f2_to_column_names_for_non_filtered(self):
        self.non_filtered_forecast_df.columns = [col + "_f2" if col != 'county' else col for col in self.non_filtered_forecast_df.columns]    
    
    def calculate_mean_for_non_filtered(self):
        grouped_df = self.non_filtered_forecast_df.groupby(['county', 'origin_datetime_f2', 'forecast_datetime_f2']).mean().reset_index()
        self.non_filtered_forecast_df = grouped_df




    def process_all_operations(self):
        self.map_county_id()
        self.filter_forecast_data()
        self.generate_non_filtered_forecast()
        self.calculate_mean_for_filtered()
        self.modify_column_names_for_filtered()
        self.change_column_names_for_filtered()
        self.modify_column_names_for_non_filtered()
        self.add_f1_and_f2_to_column_names_for_non_filtered()
        self.calculate_mean_for_non_filtered()
        self.change_column_names_for_non_filtered()


In [3]:
class HistorytWeatherProcessor:
    def __init__(self, historical_weather_df, points_data_file='/kaggle/input/mapping/points_data.parquet'):
        self.points_data_file = points_data_file
        self.county_mapping = {
            "Harjumaa": 0,
            "Hiiumaa": 1,
            "Ida-Virumaa": 2,
            "Järvamaa": 3,
            "Jõgevamaa": 4,
            "Lääne-Virumaa": 5,
            "Läänemaa": 6,
            "Pärnumaa": 7,
            "Põlvamaa": 8,
            "Raplamaa": 9,
            "Saaremaa": 10,
            "Tartumaa": 11,
            None: 12,
            "Valgamaa": 13,
            "Viljandimaa": 14,
            "Võrumaa": 15,
        }
        self.read_points_gdf = pd.read_parquet(self.points_data_file)
        self.historical_weather_df = historical_weather_df

    def map_county_id(self):
        if self.historical_weather_df is not None:
            # Convert 'latitude' and 'longitude' to float32
            self.read_points_gdf[['latitude', 'longitude']] = self.read_points_gdf[['latitude', 'longitude']].astype('float32')

            # Map county IDs
            self.read_points_gdf['county'] = self.read_points_gdf['county'].map(self.county_mapping)

            # Merge on 'latitude', 'longitude', and convert them to float32
            self.historical_weather_df = pd.merge(
                self.historical_weather_df,
                self.read_points_gdf[['latitude', 'longitude', 'county']].astype('float32'),
                on=['latitude', 'longitude'],
                how='left'
            )

            # Convert 'county' to float32
            self.historical_weather_df['county'] = self.historical_weather_df['county'].astype('float32')
        else:
            print("Error: historical_weather_df is not initialized.")

    def calculate_mean_for_historical_weather(self):
        if self.historical_weather_df is not None:
            grouped_df = self.historical_weather_df.groupby(['county', 'datetime']).mean().reset_index()
            self.historical_weather_df = grouped_df
        else:
            print("Error: historical_weather_df is not initialized.")

    def process_all_operations(self):
        self.map_county_id()
        self.calculate_mean_for_historical_weather()

In [4]:
# after you fininsh check files and columns names to be idetical to the ones in the api 

import pandas as pd

class ConcatenateDataFrames:
    def __init__(self, test_df, electricity_prices_df, historical_weather_df, *additional_dfs):
        self.test_df = test_df  # Replaced train_df with test_df
        self.electricity_prices_df = electricity_prices_df
        self.historical_weather_df1 = historical_weather_df1
        self.additional_dfs = additional_dfs
        self.filtered_df = filtered_df
        self.non_filtered_df = non_filtered_df
        self.gas_prices_df = gas_prices_df
        self.client_df = client_df
        self.revealed_targets_consumption = revealed_targets_consumption
        self.revealed_targets_production = revealed_targets_production
        
    def concatenate_dfs(self):
        # Convert relevant columns to appropriate data types
        self.test_df['prediction_datetime'] = pd.to_datetime(self.test_df['prediction_datetime'])
        self.test_df['date'] = pd.to_datetime(self.test_df['prediction_datetime']).dt.date  # Add this line
        self.electricity_prices_df['forecast_date'] = pd.to_datetime(self.electricity_prices_df['forecast_date'])
        self.historical_weather_df1['datetime'] = pd.to_datetime(self.historical_weather_df1['datetime'])
        self.non_filtered_df['datetime'] = pd.to_datetime(self.non_filtered_df['datetime'])
        self.filtered_df['datetime'] = pd.to_datetime(self.filtered_df['datetime'])
        self.gas_prices_df['forecast_date'] = pd.to_datetime(self.gas_prices_df['forecast_date'])
        self.client_df['date'] = pd.to_datetime(self.client_df['date'])
        self.test_df['date'] = pd.to_datetime(self.test_df['date'])
        self.revealed_targets_consumption['datetime'] = pd.to_datetime(self.revealed_targets_consumption['datetime'])
        self.revealed_targets_production['datetime'] = pd.to_datetime(self.revealed_targets_production['datetime'])


        
        # Joining test_df and electricity_prices_df based on forecast_date and prediction_datetime with a lag of one day
        merged_df = pd.merge(
            self.test_df,  # Replaced train_df with test_df
            self.electricity_prices_df,
            left_on='prediction_datetime',
            right_on=self.electricity_prices_df['forecast_date'] + pd.DateOffset(1),
            how='left',
            suffixes=('', '_electricity_prices')
        )

        
        # Joining test_df and gas_prices_df based on forecast_date and prediction_datetime with a lag of one day
        merged_df = pd.merge(
            merged_df,
            self.gas_prices_df,
            left_on='date',
            right_on=self.gas_prices_df['forecast_date'] + pd.DateOffset(1),
            how='left',
            suffixes=('', '_gas_prices')
        )        
        
        # Drop duplicate columns after the merge
        merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
       
        # Joining merged_df and historical_weather_df based on datetime and county with a lag of two days
        merged_df = pd.merge(
            merged_df,
            self.historical_weather_df1,
            left_on=['county', 'prediction_datetime'],
            right_on=[self.historical_weather_df1['county'], self.historical_weather_df1['datetime'] + pd.DateOffset(2)],
            how='left',
            suffixes=('', '_historical_weather')

        )

        
        # Joining merged_df and filtered_df based on datetime and county with a lag of two days
        merged_df = pd.merge(
            merged_df,
            self.filtered_df,
            left_on=['county', 'prediction_datetime'],
            right_on=[self.filtered_df['county'], self.filtered_df['datetime'] ],
            how='left',
            suffixes=('', '_filtered')

        )
        
        # Joining merged_df and non_filtered_df based on datetime and county with a lag of two days
        merged_df = pd.merge(
            merged_df,
            self.non_filtered_df,
            left_on=['county', 'prediction_datetime'],
            right_on=[self.non_filtered_df['county'], self.non_filtered_df['datetime'] + pd.DateOffset(1)],
            how='left',
            suffixes=('', '_non_filtered')
        )
        
        # Joining merged_df and self.client_df based on product_type, county, is_business, and date with a lag of one day
        merged_df = pd.merge(
            merged_df,
            self.client_df,
            left_on=['product_type', 'county', 'is_business', 'date'],
            right_on=['product_type', 'county', 'is_business', self.client_df['date'] + pd.DateOffset(2)],
            how='left',
            suffixes=('', '_client_df')
        )
        
        # Joining merged_df and self.revealed_targets_production based on product_type, county, is_business, and date with a lag of one day
        merged_df = pd.merge(
            merged_df,
            self.revealed_targets_production,
            left_on=['product_type', 'county', 'is_business', 'prediction_datetime'],
            right_on=['product_type', 'county', 'is_business', self.revealed_targets_production['datetime'] + pd.DateOffset(2)],
            how='left',
            suffixes=('', 'revealed_targets_production')
        )
        # Joining merged_df and self.revealed_targets_consumption based on product_type, county, is_business, and date with a lag of one day
        merged_df = pd.merge(
            merged_df,
            self.revealed_targets_consumption,
            left_on=['product_type', 'county', 'is_business', 'prediction_datetime'],
            right_on=['product_type', 'county', 'is_business', self.revealed_targets_consumption['datetime'] + pd.DateOffset(2)],
            how='left',
            suffixes=('', 'revealed_targets_consumption')      
            
        )        
        
        
        # Drop duplicate columns after the merge
        merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]



        return merged_df



In [5]:
import numpy as np

def encode_datetime_columns(result_df):
    # Extract time, day of year, and day of week from the datetime column
    result_df['time'] = result_df['datetime'].dt.time
    result_df['day_of_year'] = result_df['datetime'].dt.dayofyear
    result_df['day_of_week'] = result_df['datetime'].dt.dayofweek

    # Extract month of the year and day of the month
    result_df['month_of_year'] = result_df['datetime'].dt.month
    result_df['day_of_month'] = result_df['datetime'].dt.day

    # Apply cyclic encoding to generate sin and cos columns
    result_df['time_sin'] = np.sin(2 * np.pi * result_df['time'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second) / 86400)
    result_df['time_cos'] = np.cos(2 * np.pi * result_df['time'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second) / 86400)

    result_df['day_of_year_sin'] = np.sin(2 * np.pi * result_df['day_of_year'] / 365)
    result_df['day_of_year_cos'] = np.cos(2 * np.pi * result_df['day_of_year'] / 365)

    result_df['day_of_week_sin'] = np.sin(2 * np.pi * result_df['day_of_week'] / 7)
    result_df['day_of_week_cos'] = np.cos(2 * np.pi * result_df['day_of_week'] / 7)

    result_df['month_of_year_sin'] = np.sin(2 * np.pi * result_df['month_of_year'] / 12)
    result_df['month_of_year_cos'] = np.cos(2 * np.pi * result_df['month_of_year'] / 12)

    result_df['day_of_month_sin'] = np.sin(2 * np.pi * result_df['day_of_month'] / 31)
    result_df['day_of_month_cos'] = np.cos(2 * np.pi * result_df['day_of_month'] / 31)

    # Drop the original datetime, time, day of year, day of week, month of the year, and day of the month columns
    result_df.drop(columns=['time', 'day_of_year', 'day_of_week', 'month_of_year', 'day_of_month'], inplace=True)

    return result_df

In [6]:
def extract_columns(result_df, selected_columns):
    # Create a new DataFrame with only the selected columns
    result_df_version = result_df[selected_columns].copy()

    return result_df_version

# Example usage:
# Assuming 'result_df' is your DataFrame and 'selected_columns' is the list of columns to extract

selected_columns = ['prediction_datetime', 'mean_price', 'euros_per_mwh', 'temperature', 'dewpoint',
                    'shortwave_radiation', 'direct_solar_radiation',
                    'diffuse_radiation', 'latitude', 'longitude', 'temperature_f1',
                    'dewpoint_f1','day_of_month_cos', 'day_of_month_sin', 'month_of_year_cos',
                    'month_of_year_sin', 'currently_scored',
                    'surface_solar_radiation_downwards_f1', 'temperature_f2', 'dewpoint_f2',
                    'direct_solar_radiation_f2', 'direct_solar_radiation_f1', 
                    'surface_solar_radiation_downwards_f2', 'eic_count', 'installed_capacity',
                    'target_production', 'target_consumption', 'time_sin', 'time_cos', 'day_of_year_sin', 'day_of_year_cos', 
                    'day_of_week_sin', 'day_of_week_cos', 'is_consumption', 'prediction_unit_id', 'row_id', 
                    'is_business', 'product_type', 'rain', 'snowfall', 'surface_pressure','snowfall_f1', 
                    'total_precipitation_f1', 'snowfall_f2', 'total_precipitation_f2', 'windspeed_10m', 'winddirection_10m',
                   '10_metre_u_wind_component_f1', '10_metre_v_wind_component_f1', '10_metre_u_wind_component_f2', '10_metre_v_wind_component_f2'
                   , 'cloudcover_low', 'cloudcover_low_f1','cloudcover_low_f2']


In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def normalize_columns(result_df, columns_to_normalize): 
    # Create a MinMaxScaler
    scaler = MinMaxScaler()

    # Normalize specified columns
    result_df[columns_to_normalize] = scaler.fit_transform(result_df[columns_to_normalize])

    return result_df

# Example usage:
# Assuming 'result_df' is your DataFrame and 'columns_to_normalize' is the list of columns to normalize

columns_to_normalize = ['mean_price', 'euros_per_mwh', 'temperature', 'dewpoint',
                          'shortwave_radiation', 'direct_solar_radiation',
                         'diffuse_radiation', 'latitude', 'longitude', 'temperature_f1',
                         'dewpoint_f1',  'direct_solar_radiation_f1',
                         'surface_solar_radiation_downwards_f1', 'temperature_f2', 'dewpoint_f2',
                          'direct_solar_radiation_f2',
                         'surface_solar_radiation_downwards_f2', 'eic_count', 'installed_capacity',
                         'target_production', 'target_consumption', 'rain', 
                        'snowfall', 'surface_pressure','snowfall_f1', 'total_precipitation_f1', 'snowfall_f2', 'total_precipitation_f2'
                       , 'windspeed_10m', 'winddirection_10m', '10_metre_u_wind_component_f1', '10_metre_v_wind_component_f1', 
                        '10_metre_u_wind_component_f2', '10_metre_v_wind_component_f2', 'cloudcover_low', 'cloudcover_low_f1', 'cloudcover_low_f2']



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
import os
import pandas as pd

def fill_nans(df):
    # Convert 'prediction_datetime' to datetime type if not already
    if not pd.api.types.is_datetime64_ns_dtype(df['prediction_datetime']):
        df['prediction_datetime'] = pd.to_datetime(df['prediction_datetime'])
    
    # Sort the DataFrame based on prediction_datetime
    df.sort_values(by=['prediction_datetime'], inplace=True)

    # Iterate over columns and fill NaN with the average of the same hour for the last week
    for column in df.columns.difference(['prediction_datetime', 'currently_scored']):
        df[column] = df.groupby(df['prediction_datetime'].dt.hour)[column].transform(lambda x: x.fillna(x.mean()))

    # Fill NaN values for boolean column 'currently_scored' with False
    df['currently_scored'].fillna(False, inplace=True)

    return df

def process_df(df):
    """Processes a DataFrame and saves it back to a CSV file with the same name."""

    df = fill_nans(df)



In [9]:
import pandas as pd

def split_csv_by_consumption(filename):
    """Splits a CSV file based on the 'is_consumption' column and returns two DataFrames.

    Args:
        filename (str): The path to the original CSV file.

    Returns:
        tuple: A tuple containing two pandas DataFrames:
            - result_df_consumption: DataFrame with rows where is_consumption == 1
            - result_df_production: DataFrame with rows where is_consumption == 0
    """

    df = filename

    # Filter data based on is_consumption values
    result_df_consumption = df[df['is_consumption'] == 1]
    result_df_production = df[df['is_consumption'] == 0]

    return result_df_consumption, result_df_production



In [10]:
def create_features_for_unit(df, unit_id):
    df_unit = df[df['prediction_unit_id'] == unit_id].copy()  # Create a copy to avoid SettingWithCopyWarning

    # Create delay features
    for delay in (1, 24, 48, 72, 168, 336, 672):
        for feature in ['target_production', 'target_consumption', 'euros_per_mwh', 'mean_price']:
            new_feature_name = f"{feature}_delay_{delay}hrs"
            df_unit.loc[:, new_feature_name] = df_unit[feature].shift(delay)

    # Create rolling window features
    for window in (24, 48, 168, 672):
        for feature in ['target_production', 'target_consumption', 'euros_per_mwh', 'mean_price']:
            new_feature_name = f"{feature}_RW_{window}hrs"
            df_unit.loc[:, new_feature_name] = df_unit[feature].rolling(window=window).mean()

    # Additional features with delays (1, 24, 48, 72)hrs and RW(24, 48, 72)hrs
    additional_features = [
        'temperature', 'dewpoint', 'shortwave_radiation', 'windspeed_10m',
        'winddirection_10m', 'rain', 'snowfall', 'surface_pressure',
        'direct_solar_radiation', 'diffuse_radiation', 'cloudcover_low'
    ]
    delays = (1, 24, 48, 72)
    windows = (24, 48, 72)
    for feature in additional_features:
        for delay in delays:
            new_feature_name = f"{feature}_delay_{delay}hrs"
            df_unit.loc[:, new_feature_name] = df_unit[feature].shift(delay)
        for window in windows:
            new_feature_name = f"{feature}_RW_{window}hrs"
            df_unit.loc[:, new_feature_name] = df_unit[feature].rolling(window=window).mean()

    # Additional features with 1hr delay and RW(24hrs)
    more_features = [
        'temperature_f1', 'dewpoint_f1', 'direct_solar_radiation_f1',
        'surface_solar_radiation_downwards_f1', 'temperature_f2', 'dewpoint_f2',
        'direct_solar_radiation_f2', 'surface_solar_radiation_downwards_f2',
        'snowfall_f1', 'total_precipitation_f1', 'snowfall_f2',
        'total_precipitation_f2', '10_metre_u_wind_component_f1',
        '10_metre_v_wind_component_f1', '10_metre_u_wind_component_f2',
        '10_metre_v_wind_component_f2', 'cloudcover_low_f1', 'cloudcover_low_f2'
    ]
    for feature in more_features:
        df_unit.loc[:, f"{feature}_delay_1hrs"] = df_unit[feature].shift(1)
        df_unit.loc[:, f"{feature}_RW_24hrs"] = df_unit[feature].rolling(window=24).mean()

    return df_unit

In [11]:
import pandas as pd

def convert_bool_columns_to_int64(dataframe):
    bool_columns = dataframe.select_dtypes(include='bool').columns
    dataframe[bool_columns] = dataframe[bool_columns].astype('int64')
    return dataframe




In [12]:
def calculate_l_espoir(year):
    if year == 2021:
        return 0
    elif year == 2022:
        return 1 
    elif year == 2023:
        return  2
    elif year == 2024:
        return  3
    else:
        return None  # Adjust this according to your needs if other years may occur

# Apply the function to create the 'l_espoir' column


In [13]:
import lightgbm as lgb

production_model = lgb.Booster(model_file="/kaggle/input/pleasebebetter/pro.txt")
consumption_model = lgb.Booster(model_file="/kaggle/input/pleasebebetter/con.txt")



/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [14]:
import pandas as pd

def update_with_new_data(
        df_client,
        df_gas_prices,
        df_electricity_prices,
        df_forecast_weather,
        df_historical_weather,
        df_target,
        df_test,
        df_new_client,
        df_new_gas_prices,
        df_new_electricity_prices,
        df_new_forecast_weather,
        df_new_historical_weather,
        df_new_target,
        df_new_test
):
    # Convert new DataFrames to Pandas DataFrames
    # List of columns to exclude
    exclude_columns = ['data_block_id']

    # Exclude 'data_block_id' from each DataFrame
    df_new_client = df_new_client[[col for col in df_client.columns if col not in exclude_columns]]
    df_new_gas_prices = df_new_gas_prices[[col for col in df_gas_prices.columns if col not in exclude_columns]]
    df_new_electricity_prices = df_new_electricity_prices[[col for col in df_electricity_prices.columns if col not in exclude_columns]]
    df_new_forecast_weather = df_new_forecast_weather[[col for col in df_forecast_weather.columns if col not in exclude_columns]]
    df_new_historical_weather = df_new_historical_weather[[col for col in df_historical_weather.columns if col not in exclude_columns]]
    df_new_target = df_new_target[[col for col in df_target.columns if col not in exclude_columns]]
    df_new_test = df_new_test[[col for col in df_test.columns if col not in exclude_columns]]

    # Convert datetime columns to datetime format
    datetime_columns = ['prediction_datetime', 'date', 'forecast_date', 'datetime', 'forecast_datetime', 'original_datetime']
    for df in [df_client,
        df_gas_prices,
        df_electricity_prices,
        df_forecast_weather,
        df_historical_weather,
        df_target,
        df_test,df_new_client, df_new_gas_prices, df_new_electricity_prices, df_new_forecast_weather, df_new_historical_weather, df_new_target, df_new_test]:
        for col in datetime_columns:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col])
                
                

    dfs_to_convert = [
    "df_client",
    "df_gas_prices",
    "df_electricity_prices",
    "df_forecast_weather",
    "df_historical_weather",
    "df_target",
    "df_test",
    "df_new_client",
    "df_new_gas_prices",
    "df_new_electricity_prices",
    "df_new_forecast_weather",
    "df_new_historical_weather",
    "df_new_target",
    "df_new_test",
    ]

    dataframes = [df_client, df_gas_prices, df_electricity_prices, df_forecast_weather, df_historical_weather, df_target, df_test,
                   df_new_client, df_new_gas_prices, df_new_electricity_prices, df_new_forecast_weather, df_new_historical_weather, df_new_target, df_new_test]

    for df in dataframes:
        df[df.select_dtypes(include=['float64']).columns] = df.select_dtypes(include=['float64']).astype('float32')
            

    # Concatenate new data with existing data and update existing DataFrames
    df_client = pd.concat([df_client, df_new_client]).drop_duplicates(subset=["date", "county", "is_business", "product_type"])
    df_gas_prices = pd.concat([df_gas_prices, df_new_gas_prices]).drop_duplicates(subset=["forecast_date"])
    df_electricity_prices = pd.concat([df_electricity_prices, df_new_electricity_prices]).drop_duplicates(subset=["forecast_date"])
    df_forecast_weather = pd.concat([df_forecast_weather, df_new_forecast_weather]).drop_duplicates(subset=["forecast_datetime", "latitude", "longitude", "hours_ahead"])
    df_historical_weather = pd.concat([df_historical_weather, df_new_historical_weather]).drop_duplicates(subset=["datetime", "latitude", "longitude"])
    df_target = pd.concat([df_target, df_new_target]).drop_duplicates(subset=["datetime", "county", "is_business", "product_type", "is_consumption"])
    df_test = pd.concat([df_test, df_new_test]).drop_duplicates(subset=["prediction_datetime", "county", "is_business", "product_type", "is_consumption"])
    
    return df_client, df_gas_prices, df_electricity_prices, df_forecast_weather, df_historical_weather, df_target, df_test



In [15]:
import pandas as pd
client_df = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/client.csv")
gas_prices_df = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv")
electricity_prices_df = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv")
forecast_weather_df = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv")
historical_weather_df = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv")
revealed_targets_df = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/train.csv")
test_df = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/train.csv")

In [16]:
# List of DataFrames
dfs = [client_df, gas_prices_df, electricity_prices_df, forecast_weather_df, historical_weather_df, revealed_targets_df, test_df]

# Column to drop
column_to_drop = 'data_block_id'

# Drop the column from each DataFrame
for df in dfs:
    if column_to_drop in df.columns:
        df.drop(column_to_drop, axis=1, inplace=True) 
 

In [17]:
# Rename 'datetime' to 'prediction_datetime' in df_test
test_df.rename(columns={'datetime': 'prediction_datetime'}, inplace=True)

# Drop 'target' column in df_test 
test_df.drop(columns=['target'], inplace=True)

test_df['currently_scored'] = 0

# If you want the new column to be of type int64 explicitly, you can cast it
test_df['currently_scored'] = test_df['currently_scored'].astype('int64')

gas_prices_df['mean_price'] = gas_prices_df[['lowest_price_per_mwh', 'highest_price_per_mwh']].mean(axis=1)
gas_prices_df = gas_prices_df.drop(['lowest_price_per_mwh', 'highest_price_per_mwh'], axis=1) 

In [18]:
import enefit 
env = enefit.make_env()
iter_test = env.iter_test()

In [19]:
import pandas as pd

def drop_first_n_days(df, date_column_name, n=3):
    # Convert the date column to datetime if it's not already in datetime format
    if not pd.api.types.is_datetime64_any_dtype(df[date_column_name]):
        df[date_column_name] = pd.to_datetime(df[date_column_name], errors='coerce')

    # Identify the earliest date
    earliest_date = df[date_column_name].min()

    # Calculate the date threshold for dropping data 
    threshold_date = earliest_date + pd.Timedelta(days=n)

    # Filter data to keep only rows with dates after the threshold date
    filtered_df = df[df[date_column_name] >= threshold_date]

    return filtered_df


In [20]:
features = [
    "target_production_delay_1hrs", "target_consumption_delay_1hrs", "euros_per_mwh_delay_1hrs", "mean_price_delay_1hrs",
    "target_production_delay_24hrs", "target_consumption_delay_24hrs", "euros_per_mwh_delay_24hrs", "mean_price_delay_24hrs",
    "target_production_delay_48hrs", "target_consumption_delay_48hrs", "euros_per_mwh_delay_48hrs", "mean_price_delay_48hrs",
    "target_production_delay_72hrs", "target_consumption_delay_72hrs", "euros_per_mwh_delay_72hrs", "mean_price_delay_72hrs",
    "target_production_delay_168hrs", "target_consumption_delay_168hrs", "euros_per_mwh_delay_168hrs", "mean_price_delay_168hrs",
    "target_production_delay_336hrs", "target_consumption_delay_336hrs", "euros_per_mwh_delay_336hrs", "mean_price_delay_336hrs",
    "target_production_delay_672hrs", "target_consumption_delay_672hrs", "euros_per_mwh_delay_672hrs", "mean_price_delay_672hrs",
    "target_production_RW_24hrs", "target_consumption_RW_24hrs", "euros_per_mwh_RW_24hrs", "mean_price_RW_24hrs",
    "target_production_RW_48hrs", "target_consumption_RW_48hrs", "euros_per_mwh_RW_48hrs", "mean_price_RW_48hrs",
    "target_production_RW_168hrs", "target_consumption_RW_168hrs", "euros_per_mwh_RW_168hrs", "mean_price_RW_168hrs",
    "target_production_RW_672hrs", "target_consumption_RW_672hrs", "euros_per_mwh_RW_672hrs", "mean_price_RW_672hrs",
    "temperature_delay_1hrs", "temperature_delay_24hrs", "temperature_delay_48hrs", "temperature_delay_72hrs",
    "temperature_RW_24hrs", "temperature_RW_48hrs", "temperature_RW_72hrs",
    "dewpoint_delay_1hrs", "dewpoint_delay_24hrs", "dewpoint_delay_48hrs", "dewpoint_delay_72hrs",
    "dewpoint_RW_24hrs", "dewpoint_RW_48hrs", "dewpoint_RW_72hrs",
    "shortwave_radiation_delay_1hrs", "shortwave_radiation_delay_24hrs", "shortwave_radiation_delay_48hrs", "shortwave_radiation_delay_72hrs",
    "shortwave_radiation_RW_24hrs", "shortwave_radiation_RW_48hrs", "shortwave_radiation_RW_72hrs",
    "windspeed_10m_delay_1hrs", "windspeed_10m_delay_24hrs", "windspeed_10m_delay_48hrs", "windspeed_10m_delay_72hrs",
    "windspeed_10m_RW_24hrs", "windspeed_10m_RW_48hrs", "windspeed_10m_RW_72hrs",
    "winddirection_10m_delay_1hrs", "winddirection_10m_delay_24hrs", "winddirection_10m_delay_48hrs", "winddirection_10m_delay_72hrs",
    "winddirection_10m_RW_24hrs", "winddirection_10m_RW_48hrs", "winddirection_10m_RW_72hrs",
    "rain_delay_1hrs", "rain_delay_24hrs", "rain_delay_48hrs", "rain_delay_72hrs",
    "rain_RW_24hrs", "rain_RW_48hrs", "rain_RW_72hrs",
    "snowfall_delay_1hrs", "snowfall_delay_24hrs", "snowfall_delay_48hrs", "snowfall_delay_72hrs",
    "snowfall_RW_24hrs", "snowfall_RW_48hrs", "snowfall_RW_72hrs",
    "surface_pressure_delay_1hrs", "surface_pressure_delay_24hrs", "surface_pressure_delay_48hrs", "surface_pressure_delay_72hrs",
    "surface_pressure_RW_24hrs", "surface_pressure_RW_48hrs", "surface_pressure_RW_72hrs",
    "direct_solar_radiation_delay_1hrs", "direct_solar_radiation_delay_24hrs", "direct_solar_radiation_delay_48hrs", "direct_solar_radiation_delay_72hrs",
    "direct_solar_radiation_RW_24hrs", "direct_solar_radiation_RW_48hrs", "direct_solar_radiation_RW_72hrs",
    "diffuse_radiation_delay_1hrs", "diffuse_radiation_delay_24hrs", "diffuse_radiation_delay_48hrs", "diffuse_radiation_delay_72hrs",
    "diffuse_radiation_RW_24hrs", "diffuse_radiation_RW_48hrs", "diffuse_radiation_RW_72hrs",
    "cloudcover_low_delay_1hrs", "cloudcover_low_delay_24hrs", "cloudcover_low_delay_48hrs", "cloudcover_low_delay_72hrs",
    "cloudcover_low_RW_24hrs", "cloudcover_low_RW_48hrs", "cloudcover_low_RW_72hrs",
    "temperature_f1_delay_1hrs", "temperature_f1_RW_24hrs",
    "dewpoint_f1_delay_1hrs", "dewpoint_f1_RW_24hrs",
    "direct_solar_radiation_f1_delay_1hrs", "direct_solar_radiation_f1_RW_24hrs",
    "surface_solar_radiation_downwards_f1_delay_1hrs", "surface_solar_radiation_downwards_f1_RW_24hrs",
    "temperature_f2_delay_1hrs", "temperature_f2_RW_24hrs",
    "dewpoint_f2_delay_1hrs", "dewpoint_f2_RW_24hrs",
    "direct_solar_radiation_f2_delay_1hrs", "direct_solar_radiation_f2_RW_24hrs",
    "surface_solar_radiation_downwards_f2_delay_1hrs", "surface_solar_radiation_downwards_f2_RW_24hrs",
    "snowfall_f1_delay_1hrs", "snowfall_f1_RW_24hrs",
    "total_precipitation_f1_delay_1hrs", "total_precipitation_f1_RW_24hrs",
    "snowfall_f2_delay_1hrs", "snowfall_f2_RW_24hrs",
    "total_precipitation_f2_delay_1hrs", "total_precipitation_f2_RW_24hrs",
    "10_metre_u_wind_component_f1_delay_1hrs", "10_metre_u_wind_component_f1_RW_24hrs",
    "10_metre_v_wind_component_f1_delay_1hrs", "10_metre_v_wind_component_f1_RW_24hrs",
    "10_metre_u_wind_component_f2_delay_1hrs", "10_metre_u_wind_component_f2_RW_24hrs",
    "10_metre_v_wind_component_f2_delay_1hrs", "10_metre_v_wind_component_f2_RW_24hrs",
    "cloudcover_low_f1_delay_1hrs", "cloudcover_low_f1_RW_24hrs",
    "cloudcover_low_f2_delay_1hrs", "cloudcover_low_f2_RW_24hrs"
]

In [21]:
%%capture 

for (
    df_new_test, 
    df_new_target, 
    df_new_client, 
    df_new_historical_weather,
    df_new_forecast_weather, 
    df_new_electricity_prices, 
    df_new_gas_prices, 
    sample_prediction,
) in iter_test:

    for df in [df_new_test, 
           df_new_target, 
           df_new_client, 
           df_new_historical_weather,
           df_new_forecast_weather, 
           df_new_electricity_prices, 
           df_new_gas_prices, 
           sample_prediction]:

    
        # Convert boolean columns to int64
        bool_columns = df.select_dtypes(include='bool').columns
        df[bool_columns] = df[bool_columns].astype('int64')
    df_new_gas_prices['mean_price'] = df_new_gas_prices[['lowest_price_per_mwh', 'highest_price_per_mwh']].mean(axis=1)
    df_new_gas_prices = df_new_gas_prices.drop(['lowest_price_per_mwh', 'highest_price_per_mwh'], axis=1) 
    df_new_test['date'] = df_new_test['prediction_datetime'].dt.date

    client_df, gas_prices_df, electricity_prices_df, forecast_weather_df, historical_weather_df, revealed_targets_df, test_df = update_with_new_data(
        client_df,
        gas_prices_df,
        electricity_prices_df,
        forecast_weather_df,
        historical_weather_df,
        revealed_targets_df,
        test_df,
        df_new_client,
        df_new_gas_prices,
        df_new_electricity_prices,
        df_new_forecast_weather,
        df_new_historical_weather,
        df_new_target,
        df_new_test,
    )
    

    dataframes = [client_df, gas_prices_df, electricity_prices_df, forecast_weather_df, historical_weather_df, revealed_targets_df, test_df]

    for df in dataframes:
        df.fillna(method='ffill', inplace=True)
    gas_prices_df['origin_date'] = pd.to_datetime(gas_prices_df['origin_date'])
    electricity_prices_df['origin_date'] = pd.to_datetime(electricity_prices_df['origin_date'])
    forecast_weather_df['origin_datetime'] = pd.to_datetime(forecast_weather_df['origin_datetime'])
    
    if (df_new_test['currently_scored'] == 1).any():
        processor = ForecastWeatherProcessor(forecast_weather_df)
        processor.process_all_operations()
        filtered_df = processor.filtered_forecast_df
        non_filtered_df = processor.non_filtered_forecast_df
        processor = HistorytWeatherProcessor(historical_weather_df)
        processor.process_all_operations()
        historical_weather_df1 = processor.historical_weather_df
        revealed_targets_production, revealed_targets_consumption = split_revealed_targets(revealed_targets_df)
        concatenator = ConcatenateDataFrames(test_df, electricity_prices_df, historical_weather_df)
        result_df = concatenator.concatenate_dfs()        
        result_df = encode_datetime_columns(result_df)
        result_df = extract_columns(result_df, selected_columns)
        result_df = normalize_columns(result_df, columns_to_normalize)
        result_df = drop_first_n_days(result_df, 'prediction_datetime', n=3)
        consumption_df, production_df = split_csv_by_consumption(result_df)

        consumption_df = consumption_df.fillna(method='ffill')
        production_df = production_df.fillna(method='ffill')
        
        unique_units = consumption_df['prediction_unit_id'].unique()
        df_con = pd.concat([create_features_for_unit(consumption_df.copy(), unit_id) for unit_id in unique_units])
        unique_units = production_df['prediction_unit_id'].unique()
        df_pro = pd.concat([create_features_for_unit(production_df.copy(), unit_id) for unit_id in unique_units])


        df_pro['l_espoir'] = df_pro['prediction_datetime'].dt.year.apply(calculate_l_espoir)        
        df_con = df_con.drop('prediction_datetime', axis=1)  
        df_pro = df_pro.drop('prediction_datetime', axis=1) 
        
        row_ids_to_keep = sample_prediction['row_id'].tolist()
        df_con_to_be_predicted = df_con[df_con['row_id'].isin(row_ids_to_keep)]
        df_pro_to_be_predicted = df_pro[df_pro['row_id'].isin(row_ids_to_keep)]
        

        
        sub_con = pd.DataFrame({'row_id': df_con_to_be_predicted['row_id'], 'target': None})
        sub_pro = pd.DataFrame({'row_id': df_pro_to_be_predicted['row_id'], 'target': None})
        
        
        #df_con_to_be_predicted.drop('prediction_unit_id', axis=1, inplace=True)
        #df_pro_to_be_predicted.drop('prediction_unit_id', axis=1, inplace=True)
        
        
        for feature in features:
            if feature in df_pro_to_be_predicted.columns:
                df_pro_to_be_predicted[feature] = 0
            if feature in df_con_to_be_predicted.columns:
                df_con_to_be_predicted[feature] = 0
        
        predictions_con = consumption_model.predict(df_con_to_be_predicted)
        predictions_pro = production_model.predict(df_pro_to_be_predicted)
        sub_con['target'] = predictions_con
        sub_pro['target'] = predictions_pro
        concatenated_df = pd.concat([sub_con, sub_pro], sort=False).sort_values(by='row_id', ascending=True).reset_index(drop=False)


        sample_prediction['target'] = 0.0
        merged_df = sample_prediction.merge(concatenated_df[['row_id', 'target']], on='row_id', how='left', suffixes=('_sample', '_concatenated'))
        sample_prediction['target'] = np.where(merged_df['target_concatenated'].notna(), merged_df['target_concatenated'], sample_prediction['target'])  
        sample_prediction['target'] = np.clip(sample_prediction['target'], 0, np.inf)

        env.predict(sample_prediction)

    else:
        # Code to run when no element in df_new_test['currently_scored'] is equal to 1
        sample_prediction['target'] = 0.0

        env.predict(sample_prediction)